In [3]:
import os
from statistics import mode
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NeighborhoodComponentsAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, average_precision_score

x_train_load = pd.read_csv('train.csv')
x_test_load = pd.read_csv('test.csv')
y_train_load = pd.read_csv('train_labels.csv')
y_train = y_train_load.loc[:, y_train_load.columns != 'sequence']
sensor_names = ["sensor_00", "sensor_01", "sensor_02", "sensor_03", "sensor_04", "sensor_05", "sensor_06",
                "sensor_07", "sensor_08", "sensor_09", "sensor_10", "sensor_11", "sensor_12"]
submission = pd.DataFrame(
    columns=["sequence"], data=x_test_load[["sequence"]].groupby(np.arange(len(x_test_load[["sequence"]])) // 60).mean())
x_train = pd.DataFrame()
x_test = pd.DataFrame()

for i in sensor_names:
    x_train[i+"_mean"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).mean()
    x_train[i+"_std"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).std()
    x_train[i+"_max"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).max()
    x_train[i+"_min"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).min()
    x_train[i+"_sum"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).sum()
    x_train[i+"_median"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).median()
    x_train[i+"_q1"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).quantile(0.25)
    x_train[i+"_q3"] = x_train_load[i].groupby(
        np.arange(len(x_train_load[i])) // 60).quantile(0.75)

    x_test[i+"_mean"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).mean()
    x_test[i+"_std"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).std()
    x_test[i+"_max"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).max()
    x_test[i+"_min"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).min()
    x_test[i+"_sum"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).sum()
    x_test[i+"_median"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).median()
    x_test[i+"_q1"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).quantile(0.25)
    x_test[i+"_q3"] = x_test_load[i].groupby(
        np.arange(len(x_test_load[i])) // 60).quantile(0.75)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)
model = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
model.fit(x_train, y_train.values.ravel())
print("Accuracy: ", np.mean(cross_val_score(
    model, x_train, y_train.values.ravel(), cv=5, n_jobs=-1)))
model.fit(x_train, y_train.values.ravel())

y_pred_train = model.predict(x_train)

print("Average precision score: ", average_precision_score(y_train, y_pred_train))
print("Roc score: ", roc_auc_score(y_train, y_pred_train))
y_pred = model.predict(x_test)
submission["state"] = y_pred
submission.to_csv('playground_knn_out.csv', index=False)
print("Done")

C:\Users\Calvin\AppData\Local\Temp/ipykernel_26408/4280376133.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  x_train[i+"_sum"] = x_train_load[i].groupby(
C:\Users\Calvin\AppData\Local\Temp/ipykernel_26408/4280376133.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  x_train[i+"_median"] = x_train_load[i].groupby(
C:\Users\Calvin\AppData\Local\Temp/ipykernel_26408/4280376133.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

Finished feature selection
Accuracy:  0.7245069467569899
Average precision score:  0.7707223302009202
Roc score:  0.8219910392577545
Done
